# Mega cool comparison of the classification to red-wine-quality-competition
https://lms.fh-kiel.de/mod/forum/discuss.php?d=54396

In [20]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from imblearn.over_sampling import *
from imblearn.under_sampling import *

from imblearn.under_sampling import AllKNN

Load Dataset

In [21]:
h = .02  # step size in the mesh

df = pd.read_csv("winequality-red.csv")
y_tmp = np.array(df[["quality"]])
y = []
for u in range(0,np.shape(y_tmp)[0]):
    #print(y[u])
    if y_tmp[u] < 5:
        y =np.concatenate((y, [0]))
    elif y_tmp[u] < 7:
        y =np.concatenate((y, [1]))
    else:
        #print("="*5)
        y =np.concatenate((y, [2]))
df2 = df.drop(["quality"], axis=1)
X = np.array(df2)
print("Original datashape: {}".format(np.shape(X)))

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

Original datashape: (1599, 11)


In [11]:
sampler = [SMOTE(), SVMSMOTE(), RandomOverSampler(), BorderlineSMOTE()]

for UPorDWNsampler in sampler:
    print("="*10)
    print(UPorDWNsampler.__class__.__name__)
    Xres, yres = UPorDWNsampler.fit_resample(X, y)
    print("Upsampled datashape: {}".format(np.shape(Xres)))
    X_train, X_test, y_train, y_test = train_test_split( Xres, yres, test_size=0.1, random_state=42)
    asd = MLPClassifier(max_iter=750 ,solver='lbfgs', activation='relu', alpha=1, hidden_layer_sizes=(128, 16), random_state=1, warm_start=False, verbose=True)
    asd.fit(X_train, y_train)
    print('****Results****')
    train_predictions = asd.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    train_predictions = asd.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))

SMOTE
Upsampled datashape: (4086, 11)
****Results****
Accuracy: 83.6186%
Log Loss: 0.9856740560791613
SVMSMOTE
Upsampled datashape: (3179, 11)
****Results****
Accuracy: 83.0189%
Log Loss: 1.1428043943845396
RandomOverSampler
Upsampled datashape: (4086, 11)
****Results****
Accuracy: 88.2641%
Log Loss: 0.7840082259221403
BorderlineSMOTE
Upsampled datashape: (4086, 11)
****Results****
Accuracy: 85.3301%
Log Loss: 0.8019545678689887


In [22]:
ovs = RandomOverSampler()
Xres, yres = ovs.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split( Xres, yres, test_size=0.1, random_state=42)

Hier wird ein Gridsearch für die besten Paramester des MLP gemacht.

In [8]:
parameters = {'solver':('adam', "sgd", "lbfgs"), 
              'activation':('identity', 'logistic', 'tanh', 'relu'),
              'hidden_layer_sizes':[(128, 16)], 
              #'learning_rate':('constant', 'invscaling', 'adaptive'),
              'alpha':(1, 0.0001),
              'warm_start':[True]}
mlp =  MLPClassifier()
clf = GridSearchCV(mlp, parameters, n_jobs=-1)
clf.fit(X_train, y_train)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for place, tscore, params in zip(clf.cv_results_['rank_test_score'], clf.cv_results_['mean_test_score'], clf.cv_results_['params']):
    if place <= 5:
        print("Rank {} mit {}, bei folgenden parametern: {}".format(place, tscore, params))

Rank 1 mit 0.878434265010352, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (128, 16), 'solver': 'lbfgs', 'warm_start': True}
Rank 4 mit 0.8566740609287194, bei folgenden parametern: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 16), 'solver': 'lbfgs', 'warm_start': True}
Rank 2 mit 0.876800133096717, bei folgenden parametern: {'activation': 'relu', 'alpha': 1, 'hidden_layer_sizes': (128, 16), 'solver': 'lbfgs', 'warm_start': True}
Rank 5 mit 0.8558618012422359, bei folgenden parametern: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 16), 'solver': 'adam', 'warm_start': True}
Rank 3 mit 0.8672833481218575, bei folgenden parametern: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 16), 'solver': 'lbfgs', 'warm_start': True}


In [9]:
parameters = {'solver':['lbfgs'], 
              'activation':['tanh'],
              'hidden_layer_sizes':[(128, 16)], 
              #'learning_rate':['adaptive'],
              'alpha':[1],
              'warm_start':[True],
            'max_iter':[200,250,500,750, 1000]}
mlp =  MLPClassifier()
clf = GridSearchCV(mlp, parameters, n_jobs=-1)
clf.fit(X_train, y_train)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for place, tscore, params in zip(clf.cv_results_['rank_test_score'], clf.cv_results_['mean_test_score'], clf.cv_results_['params']):
    if place <= 5:
        print("Rank {} mit {}, bei folgenden parametern: {}".format(place, tscore, params))

Rank 1 mit 0.8912163561076605, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (128, 16), 'max_iter': 200, 'solver': 'lbfgs', 'warm_start': True}
Rank 5 mit 0.8806048506359065, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (128, 16), 'max_iter': 250, 'solver': 'lbfgs', 'warm_start': True}
Rank 3 mit 0.8838723750369712, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (128, 16), 'max_iter': 500, 'solver': 'lbfgs', 'warm_start': True}
Rank 2 mit 0.8868655723158829, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (128, 16), 'max_iter': 750, 'solver': 'lbfgs', 'warm_start': True}
Rank 4 mit 0.8816943951493641, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (128, 16), 'max_iter': 1000, 'solver': 'lbfgs', 'warm_start': True}


In [11]:
parameters = {'solver':['lbfgs'], 
              'activation':['tanh'],
              'hidden_layer_sizes':[(32, 2), (32, 8),(128, 8), (128, 16), (256, 16)], 
              #'learning_rate':['adaptive'],
              'alpha':[1],
              'warm_start':[True],
            'max_iter':[200]}
mlp =  MLPClassifier()
clf = GridSearchCV(mlp, parameters, n_jobs=-1)
clf.fit(X_train, y_train)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for place, tscore, params in zip(clf.cv_results_['rank_test_score'], clf.cv_results_['mean_test_score'], clf.cv_results_['params']):
    if place <= 5:
        print("Rank {} mit {}, bei folgenden parametern: {}".format(place, tscore, params))

Rank 5 mit 0.7908625406684413, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (32, 2), 'max_iter': 200, 'solver': 'lbfgs', 'warm_start': True}
Rank 4 mit 0.8501519520851819, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (32, 8), 'max_iter': 200, 'solver': 'lbfgs', 'warm_start': True}
Rank 3 mit 0.8637503697131027, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (128, 8), 'max_iter': 200, 'solver': 'lbfgs', 'warm_start': True}
Rank 1 mit 0.8855039189588879, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (128, 16), 'max_iter': 200, 'solver': 'lbfgs', 'warm_start': True}
Rank 2 mit 0.8844191807157646, bei folgenden parametern: {'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (256, 16), 'max_iter': 200, 'solver': 'lbfgs', 'warm_start': True}


In [12]:
asd = MLPClassifier(max_iter=200 ,
                    solver='lbfgs', 
                    activation='tanh', 
                    learning_rate="adaptive", 
                    alpha=1, 
                    hidden_layer_sizes=(128, 16), 
                    random_state=1, warm_start=False, verbose=True)
asd.fit(X_train, y_train)
print('****Results****')
train_predictions = asd.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
train_predictions = asd.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))

****Results****
Accuracy: 86.3081%
Log Loss: 0.4900513733239912


Search with others:

In [23]:
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X_train, y_train)

datasets = [make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1),
            linearly_separable
            ]

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, probability=True),
    SVC(gamma=2, C=1, probability=True),
    SVC(C=1000, gamma=1, kernel='rbf', probability=True),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(max_iter=750 ,solver='lbfgs', activation='relu', alpha=1, hidden_layer_sizes=(128, 16), random_state=1, warm_start=False, verbose=True),
    MLPClassifier(max_iter=200 ,solver='lbfgs',activation='tanh', learning_rate="adaptive",alpha=1, hidden_layer_sizes=(128, 16), 
                    random_state=1, warm_start=False, verbose=True),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Results****
Accuracy: 93.9394%
Log Loss: 1.01083453716749
SVC
****Results****
Accuracy: 66.4141%
Log Loss: 0.7349104346135595
SVC
****Results****
Accuracy: 100.0000%
Log Loss: 0.002596915960816789
SVC
****Results****
Accuracy: 100.0000%
Log Loss: 0.005212715195134082
DecisionTreeClassifier
****Results****
Accuracy: 76.5152%
Log Loss: 0.5335394187198177
RandomForestClassifier
****Results****
Accuracy: 78.2828%
Log Loss: 0.6258252277995481
MLPClassifier
****Results****
Accuracy: 97.7273%
Log Loss: 0.07982780923179084
MLPClassifier
****Results****
Accuracy: 95.2020%
Log Loss: 0.22680281400958213
MLPClassifier
****Results****
Accuracy: 83.5859%
Log Loss: 0.43161081324683653
AdaBoostClassifier
****Results****
Accuracy: 71.2121%
Log Loss: 0.9969069704808532
GaussianNB
****Results****
Accuracy: 65.1515%
Log Loss: 1.125465129702411
QuadraticDiscriminantAnalysis
****Results****
Accuracy: 70.9596%
Log Loss: 0.8200447519117497


Gridsearch for alternatic SVM

In [15]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1, 0.1, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                    #{'kernel': ['poly'], 'C': [1, 10, 100, 1000], 'degree':[1,2,5,10,15,20], 'coef0':[0.001, 0.01, 0.1, 1]}
                   ]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score, n_jobs=-1
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 100, 'gamma': 1, 'kernel': 'rbf'}

Grid scores on development set:

0.899 (+/-0.010) for {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
0.757 (+/-0.024) for {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.469 (+/-0.034) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.315 (+/-0.038) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.904 (+/-0.016) for {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
0.843 (+/-0.016) for {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
0.527 (+/-0.041) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.469 (+/-0.031) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.904 (+/-0.015) for {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
0.868 (+/-0.020) for {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
0.598 (+/-0.037) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.510 (+/-0.038) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.904 (+/-0.015) for {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
0.870 (+/-0.020) for {'